In [1]:
# import tensorflow as tf
import numpy as np
import h2o
import os
import pandas as pd

# from .feature_engineering_filter import Find_correct_port

In [2]:
train_data = pd.read_csv("../Datasets/ais_train.csv", delimiter="|")
test_data = pd.read_csv("../Datasets/ais_test.csv", delimiter=",")

In [3]:
vessel_data = pd.read_csv("../Datasets/vessels.csv", delimiter="|")
port_data = pd.read_csv("../Datasets/ports.csv", delimiter="|")

In [4]:
train_data["time"] = pd.to_datetime(train_data["time"])
test_data["time"] = pd.to_datetime(test_data["time"])

In [5]:
train_data=train_data.merge(vessel_data[['vesselId', 'shippingLineId']], on='vesselId', how='left')

In [6]:
port_data_renamed=pd.DataFrame()
port_data_renamed[["portId","port_latitude","port_longitude"]]=port_data[["portId","latitude","longitude"]]
train_data=train_data.merge(port_data_renamed, on="portId", how="left")

In [7]:
print(train_data.columns)

Index(['time', 'cog', 'sog', 'rot', 'heading', 'navstat', 'etaRaw', 'latitude',
       'longitude', 'vesselId', 'portId', 'shippingLineId', 'port_latitude',
       'port_longitude'],
      dtype='object')


In [ ]:
train_data_preprocessed = train_data
train_data_preprocessed.loc[train_data_preprocessed["cog"] >= 360, "cog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["sog"] >= 1023, "sog"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["rot"] == -128, "rot"] = np.nan
train_data_preprocessed.loc[train_data_preprocessed["heading"] == 511, "heading"] = (
    np.nan
)


pattern = r"^\d{2}-\d{2} \d{2}:\d{2}$"
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].where(
    train_data_preprocessed["etaRaw"].str.match(pattern, na=False), np.nan
)


train_data_preprocessed = train_data_preprocessed.sort_values("time")

print(train_data_preprocessed.head())


train_data_preprocessed = (
    train_data_preprocessed.groupby("vesselId")
    .apply(lambda group: group.ffill().bfill())
    .reset_index(drop=True)
)


print(train_data_preprocessed.head())

train_data_preprocessed["heading"] = train_data_preprocessed["heading"].fillna(0)

train_data_preprocessed = train_data_preprocessed.dropna().reset_index(drop=True)


# Replace '00-' in etaRaw with the corresponding month and day from the 'time' column
train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("00-", na=False),
    "01" + train_data_preprocessed["etaRaw"].str[2:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("-00", na=False),
    train_data_preprocessed["etaRaw"].str[:2]
    + "-01"
    + train_data_preprocessed["etaRaw"].str[5:],
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains(":60", na=False),
    train_data_preprocessed["etaRaw"].str[:9] + "59",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("60:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "01:00",
)

train_data_preprocessed["etaRaw"] = train_data_preprocessed["etaRaw"].mask(
    train_data_preprocessed["etaRaw"].str.contains("24:", na=False),
    train_data_preprocessed["etaRaw"].str[:6] + "23:59",
)


train_data_preprocessed["etaRaw"] = pd.to_datetime(
    train_data_preprocessed["time"].dt.year.astype(str)
    + "-"
    + train_data_preprocessed["etaRaw"]
    + ":00",
    format="%Y-%m-%d %H:%M:%S",
)


train_data_preprocessed["seconds_to_eta"] = (
    train_data_preprocessed["etaRaw"] - train_data_preprocessed["time"]
).dt.total_seconds()

train_data_preprocessed = train_data_preprocessed.drop(columns=["etaRaw"])

                 time    cog   sog  rot  heading  navstat       etaRaw  \
0 2024-01-01 00:00:25  284.0   0.7  0.0     88.0        0  01-09 23:00   
1 2024-01-01 00:00:36  109.6   0.0 -6.0    347.0        1  12-29 20:00   
2 2024-01-01 00:01:45  111.0  11.0  0.0    112.0        0  01-02 09:00   
3 2024-01-01 00:03:11   96.4   0.0  0.0    142.0        1  12-31 20:00   
4 2024-01-01 00:03:51  214.0  19.7  0.0    215.0        0  01-25 12:00   

   latitude  longitude                  vesselId                    portId  \
0 -34.74370  -57.85130  61e9f3a8b937134a3c4bfdf7  61d371c43aeaecc07011a37f   
1   8.89440  -79.47939  61e9f3d4b937134a3c4bff1f  634c4de270937fc01c3a7689   
2  39.19065  -76.47567  61e9f436b937134a3c4c0131  61d3847bb7b7526e1adf3d19   
3 -34.41189  151.02067  61e9f3b4b937134a3c4bfe77  61d36f770a1807568ff9a126   
4  35.88379   -5.91636  61e9f41bb937134a3c4c0087  634c4de270937fc01c3a74f3   

             shippingLineId  port_latitude  port_longitude  
0  61ec65aea8cafc0e93f0e9

In [9]:
train_latitude_radians = np.deg2rad(train_data_preprocessed["latitude"])
train_longitude_radians = np.deg2rad(train_data_preprocessed["longitude"])
train_cog_radians = np.deg2rad(train_data_preprocessed["cog"])
train_heading_radians = np.deg2rad(train_data_preprocessed["heading"])

port_latitude_radians = np.deg2rad(train_data_preprocessed["port_latitude"])
port_longitude_radians = np.deg2rad(train_data_preprocessed["port_longitude"])

train_hour = np.deg2rad(train_data_preprocessed["time"].dt.hour * 360 / 24)
train_day = np.deg2rad(train_data_preprocessed["time"].dt.day * 360 / 30)
train_month = np.deg2rad(train_data_preprocessed["time"].dt.month * 360 / 12)


train_latitude_sin = np.sin(train_latitude_radians)
train_latitude_cos = np.cos(train_latitude_radians)
train_longitude_sin = np.sin(train_longitude_radians)
train_longitude_cos = np.cos(train_longitude_radians)

port_latitude_sin = np.sin(port_latitude_radians)
port_latitude_cos = np.cos(port_latitude_radians)
port_longitude_sin = np.sin(port_longitude_radians)
port_longitude_cos = np.cos(port_longitude_radians)

train_cog_sin = np.sin(train_cog_radians)
train_cog_cos = np.cos(train_cog_radians)

train_heading_sin = np.sin(train_heading_radians)
train_heading_cos = np.cos(train_heading_radians)

train_hour_sin = np.sin(train_hour)
train_hour_cos = np.cos(train_hour)

train_day_sin = np.sin(train_day)
train_day_cos = np.cos(train_day)

train_month_sin = np.sin(train_month)
train_month_cos = np.cos(train_month)


train_data_preprocessed["latitude_sin"] = train_latitude_sin
train_data_preprocessed["latitude_cos"] = train_latitude_cos
train_data_preprocessed["longitude_sin"] = train_longitude_sin
train_data_preprocessed["longitude_cos"] = train_longitude_cos
train_data_preprocessed["port_latitude_sin"] = train_latitude_sin
train_data_preprocessed["port_latitude_cos"] = train_latitude_cos
train_data_preprocessed["port_longitude_sin"] = train_longitude_sin
train_data_preprocessed["port_longitude_cos"] = train_longitude_cos
train_data_preprocessed["cog_sin"] = train_cog_sin
train_data_preprocessed["cog_cos"] = train_cog_cos
train_data_preprocessed["heading_sin"] = train_heading_sin
train_data_preprocessed["heading_cos"] = train_heading_cos

train_data_preprocessed["hour_sin"] = train_hour_sin
train_data_preprocessed["hour_cos"] = train_hour_cos
train_data_preprocessed["day_sin"] = train_day_sin
train_data_preprocessed["day_cos"] = train_day_cos
train_data_preprocessed["month_sin"] = train_month_sin
train_data_preprocessed["month_cos"] = train_month_cos


train_data_preprocessed["cog_sog_sin"] = train_data_preprocessed["cog_sin"]*train_data_preprocessed["sog"]
train_data_preprocessed["cog_sog_cos"] = train_data_preprocessed["cog_cos"]*train_data_preprocessed["sog"]

train_data_preprocessed = train_data_preprocessed.drop(
    columns=["latitude", "longitude", "cog", "heading", "portId","cog_sin","cog_cos","sog"], axis=1
)
print(train_data_preprocessed.head(10))

                 time  rot  navstat                  vesselId  \
0 2024-01-12 14:07:47 -6.0        0  61e9f38eb937134a3c4bfd8b   
1 2024-01-12 14:31:00  5.0        0  61e9f38eb937134a3c4bfd8b   
2 2024-01-12 14:57:23  5.0        0  61e9f38eb937134a3c4bfd8b   
3 2024-01-12 15:18:48  6.0        0  61e9f38eb937134a3c4bfd8b   
4 2024-01-12 15:39:47  7.0        0  61e9f38eb937134a3c4bfd8b   
5 2024-01-12 15:54:48  5.0        0  61e9f38eb937134a3c4bfd8b   
6 2024-01-12 16:14:59 -6.0        0  61e9f38eb937134a3c4bfd8b   
7 2024-01-12 16:35:24  2.0        0  61e9f38eb937134a3c4bfd8b   
8 2024-01-12 16:55:24 -1.0        0  61e9f38eb937134a3c4bfd8b   
9 2024-01-12 17:14:36  6.0        0  61e9f38eb937134a3c4bfd8b   

             shippingLineId  port_latitude  port_longitude  seconds_to_eta  \
0  61a8e672f9cba188601e84ab      13.263333       80.341111       -374867.0   
1  61a8e672f9cba188601e84ab      18.941944       72.885278        205140.0   
2  61a8e672f9cba188601e84ab      18.941944       7

In [10]:
print(train_data_preprocessed[["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]].describe())

       latitude_sin  latitude_cos  longitude_sin  longitude_cos
count  1.522065e+06  1.522065e+06   1.522065e+06   1.522065e+06
mean   5.674148e-01  7.325459e-01   4.657097e-02   5.090908e-01
std    3.533228e-01  1.287637e-01   5.297501e-01   6.767740e-01
min   -7.376648e-01  3.328656e-01  -1.000000e+00  -9.997826e-01
25%    5.666483e-01  6.245345e-01  -9.085523e-02  -9.307284e-02
50%    6.721562e-01  7.403239e-01   7.383488e-02   9.743727e-01
75%    7.809972e-01  8.197004e-01   3.145059e-01   9.969210e-01
max    9.429743e-01  1.000000e+00   9.923086e-01   1.000000e+00


In [11]:
def Last_known_location_training_data(
    data: pd.DataFrame, shift_lenghts
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data

    """
    all_test_data = pd.DataFrame()

    for shift_length in shift_lenghts:

        grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))

        grouped_data["time_diff"] = (
            grouped_data["time"].diff(-shift_length).dt.total_seconds().abs()
        )

        original_time_and_id = grouped_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        shifted_data = grouped_data.shift(shift_length)
        shifted_data[
            [
                "last_latitude_sin",
                "last_latitude_cos",
                "last_longitude_sin",
                "last_longitude_cos",
            ]
        ] = shifted_data[
            ["latitude_sin", "latitude_cos", "longitude_sin", "longitude_cos"]
        ]

        shifted_data[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ] = original_time_and_id[
            [
                "time",
                "vesselId",
                "latitude_sin",
                "latitude_cos",
                "longitude_sin",
                "longitude_cos",
            ]
        ]

        # Drops all values with nan values
        result = shifted_data.dropna().reset_index(drop=True)

        all_test_data = pd.concat([all_test_data, result], ignore_index=True)

    # Uncomment the line below if you want to remove the "time" column after processing
    # data = data.drop("time", axis=1)

    return all_test_data

In [12]:
print(train_data_preprocessed.columns)

train_data_shifted_df = Last_known_location_training_data(
    train_data_preprocessed, [10,25]
)

# train_data_shifted_df = train_data_shifted_df.drop(columns=["time"], axis=1)

Index(['time', 'rot', 'navstat', 'vesselId', 'shippingLineId', 'port_latitude',
       'port_longitude', 'seconds_to_eta', 'latitude_sin', 'latitude_cos',
       'longitude_sin', 'longitude_cos', 'port_latitude_sin',
       'port_latitude_cos', 'port_longitude_sin', 'port_longitude_cos',
       'heading_sin', 'heading_cos', 'hour_sin', 'hour_cos', 'day_sin',
       'day_cos', 'month_sin', 'month_cos', 'cog_sog_sin', 'cog_sog_cos'],
      dtype='object')


/var/tmp/ipykernel_53551/3212926780.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))
/var/tmp/ipykernel_53551/3212926780.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  grouped_data = data.groupby("vesselId").apply(lambda x: x.sort_values("time"))


In [13]:
def append_last_known_data_test(
    test_data: pd.DataFrame, known_data: pd.DataFrame
) -> pd.DataFrame:
    """_summary_  Groups training data by vesselId, and propogates all data from last known location

    Args:
    data (_type_): _description_ the data to be altered

    Returns:
        _type_:? _description_ the altered data
    """

    if not test_data["vesselId"].isin(known_data["vesselId"]).all():
        missing_vessels = test_data[
            ~test_data["vesselId"].isin(known_data["vesselId"])
        ]["vesselId"].unique()
        raise ValueError(
            f"The following vesselIds are missing in known_data: {missing_vessels}"
        )
    print(
        test_data[~test_data["vesselId"].isin(known_data["vesselId"])][
            "vesselId"
        ].unique()
    )

    grouped_data = (
        known_data.sort_values("time")
        .groupby("vesselId")
        .tail(1)
        .reset_index(drop=True)
    )
    original_time = test_data[["time"]]
    test_data = test_data.drop("time", axis=1)

    result = pd.merge(test_data, grouped_data, how="left", on="vesselId")

    result["time_diff"] = (original_time["time"] - result["time"]).dt.total_seconds()

    return result

In [14]:
test_data_with_last_known_df = append_last_known_data_test(
    test_data, train_data_preprocessed
)
test_data_with_last_known_df[
    [
        "last_latitude_sin",
        "last_latitude_cos",
        "last_longitude_sin",
        "last_longitude_cos",
    ]
] = test_data_with_last_known_df[
    [
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ]
]
test_data_with_last_known_df = test_data_with_last_known_df.drop(
    columns=[
        "latitude_sin",
        "latitude_cos",
        "longitude_sin",
        "longitude_cos",
    ],
    axis=1,
)

print(test_data_with_last_known_df.head())

[]
   ID                  vesselId  scaling_factor                time  rot  \
0   0  61e9f3aeb937134a3c4bfe3d             0.3 2024-05-07 23:48:16  0.0   
1   1  61e9f473b937134a3c4c02df             0.3 2024-05-07 23:57:16  0.0   
2   2  61e9f469b937134a3c4c029b             0.3 2024-05-07 23:59:08  0.0   
3   3  61e9f45bb937134a3c4c0221             0.3 2024-05-07 23:52:34  0.0   
4   4  61e9f38eb937134a3c4bfd8d             0.3 2024-05-07 23:51:29  0.0   

   navstat            shippingLineId  port_latitude  port_longitude  \
0        5  61a8e672f9cba188601e84ac      31.140556      -81.496667   
1        5  61be24574ea00ae59d0fe388      14.808333      120.279444   
2        0  61ec6303a8cafc0e93f0e8f3      42.098889       11.780833   
3        1  61be24564ea00ae59d0fe37a     -43.606111      172.716111   
4        2  61ec94f1a8cafc0e93f0e92a      48.380556       -4.474167   

   seconds_to_eta  ...   day_cos  month_sin  month_cos  cog_sog_sin  \
0       -133396.0  ...  0.104528        0.

In [15]:
h2o.init(max_mem_size="26g")

train_data_shifted = h2o.H2OFrame(train_data_shifted_df)
test_data_with_last_known = h2o.H2OFrame(test_data_with_last_known_df)

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.24" 2024-07-16; OpenJDK Runtime Environment (build 11.0.24+8-post-Debian-2deb11u1); OpenJDK 64-Bit Server VM (build 11.0.24+8-post-Debian-2deb11u1, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.10/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /var/tmp/tmp7oof4y9s
  JVM stdout: /var/tmp/tmp7oof4y9s/h2o_jupyter_started_from_python.out
  JVM stderr: /var/tmp/tmp7oof4y9s/h2o_jupyter_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.5
H2O_cluster_version_age:,1 month and 24 days
H2O_cluster_name:,H2O_from_python_jupyter_2np633
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,26 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [16]:
train_data_shifted_without_validation, validation_data_shifted = (
    train_data_shifted.split_frame(ratios=[0.9], seed=42)   
)               

In [27]:
features_lat = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    "rot",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "shippingLineId",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
]
features_long = [
    # "vesselId",
    "cog_sog_sin",
    "cog_sog_cos",
    "rot",
    "shippingLineId",
    # "heading_sin",
    # "heading_cos",
    # "navstat",
    "time_diff",
    "seconds_to_eta",
    "last_latitude_sin",
    "last_latitude_cos",
    "last_longitude_sin",
    "last_longitude_cos",
    "port_latitude_sin",
    "port_latitude_cos",
    "port_longitude_sin",
    "port_longitude_cos",
    # "hour_sin",
    # "hour_cos",
    "day_sin",
    "day_cos",
    # "month_sin",
    # "month_cos",
    "latitude_sin",
    "latitude_cos",
]
target_long_sin = "longitude_sin"
target_long_cos = "longitude_cos"
target_lat_sin = "latitude_sin"
target_lat_cos = "latitude_cos"

In [18]:
params_lat_sin = {
    "ntrees": 200,  # Maximum number of trees
    "max_depth": 10,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.0005,  # Tolerance for early stopping
}
params_lat_cos = {
    "ntrees": 200,  # Maximum number of trees
    "max_depth": 8,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.0005,  # Tolerance for early stopping
}
params_long_sin = {
    "ntrees": 200,  # Maximum number of trees
    "max_depth": 10,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.0005,  # Tolerance for early stopping
}
params_long_cos = {
    "ntrees": 200,  # Maximum number of trees
    "max_depth": 10,  # Maximum depth of each tree
    "min_rows": 15,  # Minimum number of rows per leaf
    "learn_rate": 0.1,  # Learning rate
    "sample_rate": 0.8,  # Row sample rate per tree
    "col_sample_rate": 0.8,  # Column sample rate per tree
    "reg_lambda": 1.0,  # L2 regularization term
    "reg_alpha": 0.1,  # L1 regularization term
    "seed": 42,  # Random seed for reproducibility
    "stopping_rounds": 10,  # Early stopping rounds
    "stopping_metric": "AUTO",  # Metric for early stopping
    "stopping_tolerance": 0.0005,  # Tolerance for early stopping
}
# score_eval_metric_only if you want to predict only on the evaluation dataset, could help against overfitting

gbm_lat_sin = h2o.estimators.H2OXGBoostEstimator(**params_lat_sin)
gbm_lat_cos = h2o.estimators.H2OXGBoostEstimator(**params_lat_cos)
gbm_long_sin = h2o.estimators.H2OXGBoostEstimator(**params_long_sin)
gbm_long_cos = h2o.estimators.H2OXGBoostEstimator(**params_long_cos)


In [19]:
gbm_lat_sin.train(
    x=features_lat,
    y=target_lat_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |

/opt/conda/lib/python3.10/site-packages/h2o/estimators/estimator_base.py:192: RuntimeWarning: early stopping is enabled but neither score_tree_interval or score_each_iteration are defined. Early stopping will not be reproducible!
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729705799805_1


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0005474388110204393
RMSE: 0.02339741034859284
MAE: 0.00693779021288686
RMSLE: 0.02508570734916202
Mean Residual Deviance: 0.0005474388110204393

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.0006835261207320038
RMSE: 0.026144332478225635
MAE: 0.007364094917138825
RMSLE: 0.028054700157270504
Mean Residual Deviance: 0.0006835261207320038

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance      validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ---------------------  --------------------  ---------------------  ---------------------
     2024-10-23 17:51:55  0.063 sec          0.0                0.3595303239740979    0.27348894912264937    0.1292620538569198     0.361145329353887     0.27432289067186216    0.1304259489141275
     2024-10-23 17:52:08  13.133 sec         1.0                0.32461046263430215   0.2468269298035676     0.10537195245165569    0.3260392029410149    0.24757959400126123    0.1063015618544123
     2024-10-23 17:52:15  20.560 sec         3.0                0.26511477101572556   0.20134426256542354    0.0702858418107206     0.26626149139986577   0.20196251398855913    0.07089518180248079
     2024-10-23 17:52:22  27.486 sec         5.0                0.21709127473497475   0.16450021252980063    0.04712862156605628    0.21802499999733518   0.16501284695755072    0.04753490062383801
     2024-10-23 17:52:29  34.389 sec         7.0                0.17860205266444384   0.1347438219193816     0.03189869321595277    0.17934847088969483   0.1351701984161146     0.03216587401047172
     2024-10-23 17:52:36  40.734 sec         9.0                0.14774169905706103   0.11062495572355029    0.021827609640267188   0.14836841008016305   0.11099343140243342    0.02201318510971543
     2024-10-23 17:52:43  47.834 sec         11.0               0.1231924895854461    0.09111220305446154    0.015176389490260246   0.12371339238066398   0.09142518656451225    0.015305003454332129
     2024-10-23 17:52:50  54.856 sec         13.0               0.10374609735985703   0.07527774185287316    0.010763252717400935   0.1042222380855839    0.07554923309585419    0.010862274911568135
     2024-10-23 17:52:56  1 min  1.001 sec   15.0               0.08842234331866344   0.06247885421896535    0.007818510797963587   0.08886733514213269   0.06271635655442348    0.007897403255264132
     2024-10-23 17:53:03  1 min  8.352 sec   17.0               0.07666317465930018   0.05216998839239371    0.005877242348842365   0.07709925072169221   0.05238145301651039    0.005944294461846357
---  ---                  ---                ---                ---                   ---                    ---                    ---                   ---                    ---
     2024-10-23 17:56:45  4 min 49.949 sec   89.0               0.030924366651869574  0.008502742843521097   0.000956316452819263   0.032785676778881584  0.008784997013919434   0.001074900601849295
     2024-10-23 17:57:12  5 min 17.278 sec   98.0               0.029721187343016132  0.008284133109221742   0.0008833489770786624  0.03171591532295777   0.008586163572738526   0.0010058992847730272
     2024-10-23 17:57:44  5 min 49.319 sec   109.0              0.02898400057646589   0.008113391673411545   0.0008400722894165752  0.031062297326948353  0.008429023368809341   0.0009648663152277428
     2024-10-23 17:58:18  6 min 23.254 sec   120.0              0.028130537537087445  0.00790931881265749    0.0007913271421254857  0.030288770101058026  0.00823434225092002    0.0009174095942347467
     2024-10-23 17:58:57 

In [ ]:
gbm_lat_cos.train(
    x=features_lat,  # .append(latitude_sin)
    y=target_lat_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729705799805_2


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.00014707434711747443
RMSE: 0.012127421288859163
MAE: 0.004906695796955109
RMSLE: 0.0067067361123109145
Mean Residual Deviance: 0.00014707434711747443

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.0001531987167772356
RMSE: 0.012377346919967768
MAE: 0.004976473001395157
RMSLE: 0.006846469856594696
Mean Residual Deviance: 0.0001531987167772356

Scoring History: 
     timestamp            duration          number_of_trees    training_rmse         training_mae          training_deviance       validation_rmse       validation_mae        validation_deviance
---  -------------------  ----------------  -----------------  --------------------  --------------------  ----------------------  --------------------  --------------------  ----------------------
     2024-10-23 18:02:38  0.003 sec         0.0                0.2658095103041479    0.23339721520724913   0.07065469576813092     0.2658357670249574    0.2334561551947388    0.07066865502974744
     2024-10-23 18:02:48  10.411 sec        1.0                0.239441982509596     0.21008131696800222   0.05733246298812567     0.23946978302717062   0.21013927725308587   0.057345776983080174
     2024-10-23 18:02:52  14.534 sec        3.0                0.19440615982427564   0.1702267042708372    0.03779375497762181     0.19443910053288546   0.17028699089137148   0.03780656381603754
     2024-10-23 18:02:57  18.959 sec        5.0                0.15800075596872662   0.13796736157031708   0.024964238886689104    0.1580367540873887    0.13802538582397078   0.024975615642477772
     2024-10-23 18:03:02  23.966 sec        7.0                0.1286266296396541    0.11192174037163989   0.01654480985245674     0.12866212512980457   0.11197651242296264   0.01655394244291749
     2024-10-23 18:03:08  29.668 sec        9.0                0.10492290539350885   0.09088984102061716   0.011008816076215209    0.10495988445298515   0.09094214398613096   0.011016577344383993
     2024-10-23 18:03:12  34.059 sec        11.0               0.0858692990207485    0.0739085617686555    0.0073735365143147175   0.08590176854476628   0.07395702776940245   0.007379113839118598
     2024-10-23 18:03:17  38.783 sec        13.0               0.07058836244547002   0.060215550567538086  0.0049827169127330435   0.07061789634956527   0.0602582170639654    0.004986887284837944
     2024-10-23 18:03:21  43.380 sec        15.0               0.058349658375576006  0.049193855417651505  0.003404682632546427    0.058370434945719946  0.04923048955883918   0.003407107675752524
     2024-10-23 18:03:28  50.355 sec        18.0               0.044580654588754806  0.03666699497176756   0.001987434763561865    0.04459270974199596   0.03669501045603927   0.0019885097621339014
---  ---                  ---               ---                ---                   ---                   ---                     ---                   ---                   ---
     2024-10-23 18:06:13  3 min 35.107 sec  86.0               0.014757665730972942  0.005804993015101975  0.0002177886978271331   0.014813130813033128  0.005825448178763787  0.0002194288444840315
     2024-10-23 18:06:36  3 min 58.071 sec  96.0               0.014446599158303982  0.00569972025210614   0.0002087042272407093   0.014512275033470563  0.005725115875674686  0.00021060612664709305
     2024-10-23 18:07:00  4 min 22.514 sec  106.0              0.014133509457945368  0.005598325443609364  0.00019975608959783118  0.014222013538156294  0.005629208938131871  0.00020226566907950093
     2024-10-23 18:07:29  4 min 50.742 sec  118.0              0.013801203586465666  0.00548182367189797   0.00019047322043507273  0.01390922746973605   0.005517822122552563  0.0001934666088048599
     2024-10-23 18:07:57  5 min 19.124 sec 

In [ ]:
performance_lat_sin = gbm_lat_sin.model_performance(validation_data=validation_data_shifted)
performance_lat_cos = gbm_lat_cos.model_performance(validation_data=validation_data_shifted)


# Print the performance metrics
print(performance_lat_sin)
print(performance_lat_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.0006835261207320038
RMSE: 0.026144332478225635
MAE: 0.007364094917138825
RMSLE: 0.028054700157270504
Mean Residual Deviance: 0.0006835261207320038
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.0001531987167772356
RMSE: 0.012377346919967768
MAE: 0.004976473001395157
RMSLE: 0.006846469856594696
Mean Residual Deviance: 0.0001531987167772356


In [28]:
gbm_long_sin.train(
    x=features_long,
    y=target_long_sin,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)
#MSE: 0.00023605270678827214

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729705799805_6


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.0002435928079115762
RMSE: 0.015607460008328588
MAE: 0.0055397249411740955
RMSLE: NaN
Mean Residual Deviance: 0.0002435928079115762

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.00036365483471347996
RMSE: 0.019069736094489614
MAE: 0.005900173867516693
RMSLE: NaN
Mean Residual Deviance: 0.00036365483471347996

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-23 18:35:44  0.004 sec          0.0                0.6973162521011385    0.5570587987953262     0.48624995544437843     0.697220866820864     0.5570385456848005     0.486116937130437
     2024-10-23 18:35:56  11.696 sec         1.0                0.6282375585129145    0.5018188814224492     0.39468242992626773     0.6281696086105709    0.5018207053776345     0.3945970571819578
     2024-10-23 18:36:00  15.976 sec         2.0                0.5660888796806933    0.45210107772418434    0.32045661969814243     0.566051555937549     0.4521272641528803     0.32041436397932016
     2024-10-23 18:36:06  21.835 sec         3.0                0.510154473781825     0.4073787450349321     0.2602575871196107      0.510139100865845     0.4074170152472313     0.2602419022322128
     2024-10-23 18:36:12  27.826 sec         5.0                0.4145340189023047    0.3307790142728778     0.1718384528272963      0.4145598827268712    0.33084666607837965    0.17185989636651722
     2024-10-23 18:36:18  33.959 sec         7.0                0.3371455735915424    0.2687276093510991     0.11366713779237012     0.3372220723476981    0.26881675124440824    0.11371872607847612
     2024-10-23 18:36:24  39.954 sec         9.0                0.27467767439773616   0.2184999334826239     0.07544782481254877     0.27482241719915124   0.21860254418770578    0.07552736099518434
     2024-10-23 18:36:31  46.505 sec         11.0               0.22421037575804117   0.17781848886510823    0.05027029259756202     0.22445567965927452   0.1779432817001401     0.05038035213130686
     2024-10-23 18:36:37  53.370 sec         13.0               0.18345976443283468   0.14487756131216803    0.0336574851657512      0.18379650387467092   0.14501519636806368    0.033781154836551926
     2024-10-23 18:36:44  59.695 sec         15.0               0.1507227825583006    0.11825241950263357    0.022717357182116763    0.15118835970143138   0.11840922071928338    0.022857920109209402
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-23 18:40:33  4 min 49.115 sec   85.0               0.021231189350806087  0.007016489537690172   0.00045076340124978185  0.02413024316126843   0.0073084627458901725  0.0005822686350219418
     2024-10-23 18:40:59  5 min 15.130 sec   93.0               0.020504957257801843  0.0068685840540043     0.0004204532721442805   0.023486667561564582  0.0071679580482599415  0.00055162355314745
     2024-10-23 18:41:31  5 min 46.869 sec   103.0              0.01977374483056588   0.006671977561192495   0.00039100098462433087  0.022861578314576112  0.00698265146573473    0.0005226517630334968
     2024-10-23 18:42:02  6 min 18.310 sec   113.0              0.01923077489642888   0.006552398281731176   0.0003698227031171192   0.022362372523223945  0.006868341621196199   0.0005000757048674412
     2024-10-23 18:42:39  6 min 55.161 s

In [29]:
gbm_long_cos.train(
    x=features_long,  # .append("longitude_sin")
    y=target_long_cos,
    training_frame=train_data_shifted_without_validation,
    validation_frame=validation_data_shifted,
)

xgboost Model Build progress: |██████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OXGBoostEstimator : XGBoost
Model Key: XGBoost_model_python_1729705799805_7


Model Summary: 
    number_of_trees
--  -----------------
    200

ModelMetricsRegression: xgboost
** Reported on train data. **

MSE: 0.00013470981756651316
RMSE: 0.011606455857259492
MAE: 0.003105913857507238
RMSLE: NaN
Mean Residual Deviance: 0.00013470981756651316

ModelMetricsRegression: xgboost
** Reported on validation data. **

MSE: 0.00019859001791834753
RMSE: 0.014092197057888011
MAE: 0.0033277104937216727
RMSLE: NaN
Mean Residual Deviance: 0.00019859001791834753

Scoring History: 
     timestamp            duration           number_of_trees    training_rmse         training_mae           training_deviance       validation_rmse       validation_mae         validation_deviance
---  -------------------  -----------------  -----------------  --------------------  ---------------------  ----------------------  --------------------  ---------------------  ----------------------
     2024-10-23 18:46:43  0.002 sec          0.0                0.67669504197939      0.6008440122333621     0.45791617983948835     0.6781214557626759    0.6021292653116523     0.45984870876569084
     2024-10-23 18:46:54  11.568 sec         1.0                0.6094456505815108    0.5410542783737177     0.37142400101272094     0.6107208730242907    0.5422067894169109     0.3729799847475518
     2024-10-23 18:47:00  17.248 sec         3.0                0.49465279528855227   0.4388954935998272     0.24468138788677837     0.49568336508525235   0.43983412602031796    0.24570199842223955
     2024-10-23 18:47:07  23.936 sec         5.0                0.401737776519609     0.3561275760683088     0.16139324108291933     0.4025712882408014    0.35687616655375765    0.1620636421158584
     2024-10-23 18:47:13  29.860 sec         7.0                0.3264032982190439    0.28900666457334895    0.1065391130882701      0.3270992983654775    0.28961426108268107    0.10699395099118768
     2024-10-23 18:47:19  35.773 sec         9.0                0.2655191936865257    0.234725152997959      0.07050044221594275     0.26611589694822535   0.23521442156799452    0.0708176706085585
     2024-10-23 18:47:25  42.142 sec         11.0               0.2162785331304777    0.19073235616245504    0.046776403893071146    0.21680248848059588   0.1911242721112135     0.04700331901137891
     2024-10-23 18:47:31  48.282 sec         13.0               0.17646514440846206   0.15507416770648902    0.031139947191099367    0.1769301357787799    0.1553840424191338     0.031304272946697485
     2024-10-23 18:47:37  54.448 sec         15.0               0.14428446313317586   0.12613661369214       0.02081800630162878     0.14474930113862539   0.12639587488490958    0.02095236018012046
     2024-10-23 18:47:43  1 min  0.627 sec   17.0               0.11836618974454191   0.10270277561573968    0.014010554874640898    0.11884592497527459   0.10292245642124782    0.014124353883228595
---  ---                  ---                ---                ---                   ---                    ---                     ---                   ---                    ---
     2024-10-23 18:51:41  4 min 58.015 sec   91.0               0.015352640142369922  0.0039019507036091717  0.00023570355934110835  0.017589720297745526  0.004076521868302299   0.00030939826015292097
     2024-10-23 18:52:08  5 min 25.244 sec   99.0               0.014948540231901434  0.003808910916911812   0.00022345885506477577  0.017155691879265158  0.0039850336953968805  0.0002943177638562844
     2024-10-23 18:52:38  5 min 55.471 sec   109.0              0.014364143714066585  0.003685899404222701   0.0002063286246383586   0.016645474356927308  0.003868937590515668   0.0002770718165671246
     2024-10-23 18:53:10  6 min 27.275 sec   119.0              0.013868854729601877  0.003577016170822063   0.00019234513151080034  0.01621233882866695   0.003766999481484866   0.0002628399302955021
     2024-10-23 18:53:46  7 min  3

In [30]:
performance_long_sin = gbm_long_sin.model_performance(test_data=validation_data_shifted)
performance_long_cos = gbm_long_cos.model_performance(test_data=validation_data_shifted)

print(performance_long_sin)
print(performance_long_cos)

ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00036365483471347996
RMSE: 0.019069736094489614
MAE: 0.005900173867516693
RMSLE: NaN
Mean Residual Deviance: 0.00036365483471347996
ModelMetricsRegression: xgboost
** Reported on test data. **

MSE: 0.00019859001791834753
RMSE: 0.014092197057888011
MAE: 0.0033277104937216727
RMSLE: NaN
Mean Residual Deviance: 0.00019859001791834753


In [31]:
test_data_with_predicted_lat = test_data_with_last_known

lat_predictions_sin = gbm_lat_sin.predict(test_data_with_last_known)
test_data_with_last_known["latitude_sin"] = lat_predictions_sin
lat_predictions_cos = gbm_lat_cos.predict(test_data_with_last_known)
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

test_data_with_predicted_lat["latitude_sin"] = lat_predictions_sin
test_data_with_predicted_lat["latitude_cos"] = lat_predictions_cos

long_predictions_sin = gbm_long_sin.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_sin"] = long_predictions_sin
long_predictions_cos = gbm_long_cos.predict(test_data_with_predicted_lat)
test_data_with_last_known["longitude_cos"] = long_predictions_cos

xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%
xgboost prediction progress: |███████████████████████████████████████████████████| (done) 100%


In [38]:
# Convert sine and cosine values back to radians
lat_predictions_sin = lat_predictions_sin.as_data_frame()
lat_predictions_cos = lat_predictions_cos.as_data_frame()
long_predictions_sin = long_predictions_sin.as_data_frame()
long_predictions_cos = long_predictions_cos.as_data_frame()


lat_predictions_radians = np.arctan2(lat_predictions_sin, lat_predictions_cos)
long_predictions_radians = np.arctan2(long_predictions_sin, long_predictions_cos)

# Convert radians to degrees
lat_predictions_degrees = np.rad2deg(lat_predictions_radians)
long_predictions_degrees = np.rad2deg(long_predictions_radians)

# Print the first few rows to verify the conversion
print(lat_predictions_degrees.head())
print(long_predictions_degrees.head())

AttributeError: 'DataFrame' object has no attribute 'as_data_frame'

In [33]:
def create_prediction_visualization_data(validation_data):
    lat_val_sin = gbm_lat_sin.predict(validation_data)
    lat_val_cos = gbm_lat_cos.predict(validation_data)
    long_val_sin = gbm_long_sin.predict(validation_data)
    long_val_cos = gbm_long_cos.predict(validation_data)

    lat_val_sin = lat_val_sin.as_data_frame()
    lat_val_cos = lat_val_cos.as_data_frame()
    long_val_sin = long_val_sin.as_data_frame()
    long_val_cos = long_val_cos.as_data_frame()

    validation_data = validation_data.as_data_frame()

    lat_val_radians = np.arctan2(lat_val_sin, lat_val_cos)
    long_val_radians = np.arctan2(long_val_sin, long_val_cos)

    evaluation_lat_radians = np.arctan2(
        validation_data["latitude_sin"], validation_data["latitude_cos"]
    )
    evaluation_long_radians = np.arctan2(
        validation_data["longitude_sin"], validation_data["longitude_cos"]
    )

    # Convert radians to degrees
    lat_val_degrees = np.rad2deg(lat_val_radians)
    long_val_degrees = np.rad2deg(long_val_radians)

    evaluation_lat_degrees = np.rad2deg(evaluation_lat_radians)
    evaluation_long_degrees = np.rad2deg(evaluation_long_radians)

    eval_predictions = pd.concat([lat_val_degrees, long_val_degrees], axis=1)

    eval_actual = pd.concat([evaluation_lat_degrees, evaluation_long_degrees], axis=1)

    eval_predictions.columns = ["latitude_predicted", "longitude_predicted"]
    eval_actual.columns = ["latitude", "longitude"]

    eval = pd.DataFrame()
    eval[["latitude_predicted", "longitude_predicted"]] = eval_predictions
    eval[["latitude", "longitude"]] = eval_actual
    eval[["vesselId", "time"]] = validation_data[["vesselId", "time"]]
    eval.to_csv("eval_predictions.csv")


# create_prediction_visualization_data(validation_data_shifted)

In [39]:
predictions = pd.concat([lat_predictions_degrees, long_predictions_degrees], axis=1)
predictions.columns = ["latitude_predicted", "longitude_predicted"]

In [40]:
predictions["ID"] = test_data["ID"]
predictions = predictions[["ID", "longitude_predicted", "latitude_predicted"]]

In [41]:
print(predictions.columns)

Index(['ID', 'longitude_predicted', 'latitude_predicted'], dtype='object')


In [42]:
predictions.to_csv("predictions.csv", index=False)